In [91]:
# import dependencies
import requests
import json
import os
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [92]:
# set up api key and headers
api_key = os.environ['DOMINO_USER_API_KEY']

# api host
host = os.environ['DOMINO_API_HOST']

In [93]:
# get projects
project_fetch_headers = {'X-Domino-Api-Key': api_key}
projects_url = '{host}/api/projects/beta/projects?limit=500'.format(host=host)
projects = requests.get(projects_url, headers=project_fetch_headers)

In [94]:
# get models
model_fetch_headers = {'X-Domino-Api-Key': api_key}
all_models = []
for project in projects.json()['projects']:
    project_id = project['id']
    models_url = '{host}/v4/modelManager/getModels?projectId={project_id}'.format(host=host, project_id=project_id)
    models = requests.get(models_url, headers=model_fetch_headers)
    if models.status_code == 200:
        for model in models.json():
            all_models.append(model)

In [95]:
# get model invocations
for model in all_models:
    model_activity_url = '{host}/v4/modelManager/{version_id}/getLast24HoursInvocationCounts'.format(host=host, version_id=model['activeModelVersionId'])
    model_activity = requests.get(model_activity_url, headers=model_fetch_headers)
    model_24h_invocations = 0
    for code in model_activity.json():
        model_24h_invocations += code['invocationCount']
    model['oneday_invocations'] = model_24h_invocations

In [103]:
all_models_df = pd.DataFrame.from_records(all_models)
running_models_df = all_models_df[all_models_df.activeVersionStatus == 'Running']
running_models_df = running_models_df.drop(columns=['description', 'activeVersionDataPlaneId', 'lastModified', 'projectId', 'activeModelVersionId', 'isAsync'])
active_models_df = running_models_df[running_models_df.oneday_invocations != 0]
inactive_models_df = running_models_df[running_models_df.oneday_invocations == 0]

In [104]:
running_models_df

,id,name,activeVersionNumber,activeVersionStatus,projectName,projectOwnerUsername,owners,oneday_invocations
3,625478144906b439238b5cdc,Auto MPG Prediction,3,Running,Auto-MPG-Prediction-Model,ModelOps,"[{'id': '61e9d8b873e0000e1f1e4712', 'username': 'jaclyn_patterson', 'fullName': 'Jaclyn Patterson'}]",242
26,63f889a99fb0fd477f3a599e,Credit-Risk-Monitor,8,Running,CreditRisk,bryan_prosser,"[{'id': '61f7e9025ea2aa19e985e378', 'username': 'bryan_prosser', 'fullName': 'Bryan Prosser'}]",480
32,65b04f6b1266902edb95b25e,Quickstart-DMM-API,1,Running,DMM-Quickstart,dave_heinicke,"[{'id': '62be04849c2ba24610ffefc0', 'username': 'dave_heinicke', 'fullName': 'Dave Heinicke'}]",0
45,65e5bb742a47b32f7f5af564,Wine Model 2,1,Running,MLOps-Workshop-Bryan,bryan_prosser,"[{'id': '61f7e9025ea2aa19e985e378', 'username': 'bryan_prosser', 'fullName': 'Bryan Prosser'}]",0
46,629ee30bc73079282fa26f28,ManufacturingDemo Model,3,Running,ManufacturingDemo,andrea_lowe,"[{'id': '6258610c4906b439238b6aea', 'username': 'andrea_lowe', 'fullName': 'Andrea Lowe'}]",0
71,62574492ec229400bbd3819b,Stock Price Prediction,1,Running,Stock-Price-Estimator,ModelOps,"[{'id': '61e9d8b873e0000e1f1e4712', 'username': 'jaclyn_patterson', 'fullName': 'Jaclyn Patterson'}]",28222


In [105]:
inactive_models_df

,id,name,activeVersionNumber,activeVersionStatus,projectName,projectOwnerUsername,owners,oneday_invocations
32,65b04f6b1266902edb95b25e,Quickstart-DMM-API,1,Running,DMM-Quickstart,dave_heinicke,"[{'id': '62be04849c2ba24610ffefc0', 'username': 'dave_heinicke', 'fullName': 'Dave Heinicke'}]",0
45,65e5bb742a47b32f7f5af564,Wine Model 2,1,Running,MLOps-Workshop-Bryan,bryan_prosser,"[{'id': '61f7e9025ea2aa19e985e378', 'username': 'bryan_prosser', 'fullName': 'Bryan Prosser'}]",0
46,629ee30bc73079282fa26f28,ManufacturingDemo Model,3,Running,ManufacturingDemo,andrea_lowe,"[{'id': '6258610c4906b439238b6aea', 'username': 'andrea_lowe', 'fullName': 'Andrea Lowe'}]",0
